This notebook is an introduction to how we can use YOLOv3 to process images

First, verify that all dependancies are satisfied:

In [1]:
!pip install opencv-python
!pip install matplotlib

In [2]:
import src.network as network
import src.file_helper as fh
import youtube_dl

weights = "yolov3.weights"
cfg = "yolov3.cfg"
classes = "coco.names"

image = "demo.jpg"

net = network.load(weights, cfg)
classes = network.prepare_classes(classes)

output_layers = network.extract_output_layers(net)

img = fh.load_image(image)

blob = network.image_to_blob(img, 1/255)

outputs = network.forward(net, blob, output_layers)

In [2]:
processed_image = network.postprocess(img, outputs, classes, 0)

fh.show_image(processed_image)

1008
[[177]
 [165]
 [183]
 [152]
 [144]]
0.9529231190681458
0.8640747666358948
0.837643027305603
0.7424910068511963
0.6239824891090393


In [16]:
print(img.shape)

(371, 660, 3)
